In [39]:
#PigLab1NoteBook

! pip install pandas
! pip install openpyxl
!pip install rdflib
import pandas as pd
import rdflib
import hashlib
import numpy as np
import urllib.parse
from rdflib import Literal, Namespace, RDF, URIRef, Graph, RDFS, OWL
from rdflib.namespace import FOAF, XSD
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


### Step 01: Define the file paths and Load the data into Pandas dataframes and Clean

In [40]:
# Define the file paths
files = ['data/PigData/Lab1/DECIDE-BIOFAR.xlsx']

# Load the data into Pandas dataframes
dfs = []
for file in files:
    df = pd.read_excel(file, engine='openpyxl')
    dfs.append(df)

df = dfs[0]


In [41]:
df

,Id,Provincia,Fecha,Patógeno,Resultado,Muestra,Edad,Granja
0,S-21-0055,?,2021-01-05,SIV,POSITIVO,Pulmón,NaN,NaN
1,S-21-0058,Huesca,2021-01-05,SIV,NEGATIVO,Lechón,Lechones abortados,NaN
2,S-21-0062,NaN,2021-01-05,SIV,NEGATIVO,Pulmón,NaN,NaN
3,S-21-0067,NaN,2021-01-05,SIV,NEGATIVO,Pulmón,NaN,NaN
4,S-21-0102,?,2021-01-08,SIV,NEGATIVO,Fluido oral,NaN,NaN
...,...,...,...,...,...,...,...,...
1039,S-23-9748,Huesca,2023-10-24,Lawsonia intracellularis,POSITIVO,Heces,12 semanas de vida,NaN
1040,S-23-9778,NaN,2023-10-25,Mycoplasma hyopneumoniae,NEGATIVO,Pulmón,NaN,NaN
1041,S-23-9852,NaN,2023-10-26,Mycoplasma hyopneumoniae,NEGATIVO,Pulmón,NaN,NaN
1042,S-23-9921,NaN,2023-10-27,Mycoplasma hyopneumoniae,NEGATIVO,Pulmón,Transició,NaN


In [42]:
#Determine the PigStage by Age, Age Unit, FarmType. and map to one coulumn


df = df.rename(columns={
    'Id': 'SampleNumber',
    'Provincia': 'Province',
    'Fecha ': 'Date',
    'Patógeno': 'Pathogen',
    'Resultado': 'Result',
    'Edad': 'Age', #Age of animal
    'Muestra': 'SampleType',
    'Granja': 'FarmType', #age of animal
})

# Set DiagnosticTest to "PCR" Country to "Spain", Breed to "Meat", and LabReference to "1".
df['DiagnosticTest'] = 'PCR'
df['Country'] = 'Spain'
df['Breed'] = 'Meat'
df["Lab_Reference"] = "1"






In [43]:
df

,SampleNumber,Province,Date,Pathogen,Result,SampleType,Age,FarmType,DiagnosticTest,Country,Breed,Lab_Reference
0,S-21-0055,?,2021-01-05,SIV,POSITIVO,Pulmón,NaN,NaN,PCR,Spain,Meat,1
1,S-21-0058,Huesca,2021-01-05,SIV,NEGATIVO,Lechón,Lechones abortados,NaN,PCR,Spain,Meat,1
2,S-21-0062,NaN,2021-01-05,SIV,NEGATIVO,Pulmón,NaN,NaN,PCR,Spain,Meat,1
3,S-21-0067,NaN,2021-01-05,SIV,NEGATIVO,Pulmón,NaN,NaN,PCR,Spain,Meat,1
4,S-21-0102,?,2021-01-08,SIV,NEGATIVO,Fluido oral,NaN,NaN,PCR,Spain,Meat,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1039,S-23-9748,Huesca,2023-10-24,Lawsonia intracellularis,POSITIVO,Heces,12 semanas de vida,NaN,PCR,Spain,Meat,1
1040,S-23-9778,NaN,2023-10-25,Mycoplasma hyopneumoniae,NEGATIVO,Pulmón,NaN,NaN,PCR,Spain,Meat,1
1041,S-23-9852,NaN,2023-10-26,Mycoplasma hyopneumoniae,NEGATIVO,Pulmón,NaN,NaN,PCR,Spain,Meat,1
1042,S-23-9921,NaN,2023-10-27,Mycoplasma hyopneumoniae,NEGATIVO,Pulmón,Transició,NaN,PCR,Spain,Meat,1


In [44]:
#Annotations of different terminologies


# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "POSITIVO": "1",
    "NEGATIVO": "0",
}
df["Result"] = df["Result"].map(resultMapping).fillna("Missing")




# Add Pathogen mapping for annotation
PathogenMapping = {
    "M. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae antistoffen" : "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "SIV" : "Swine influenza virus",
    "Mycoplasma hyopneumoniae": "Mycoplasma hyopneumoniae",
     "Lawsonia intracellularis": "Lawsonia intracellularis",
    
}
df["Pathogen"] = df["Pathogen"].map(PathogenMapping).fillna("Missing")



# SampleTypeMapping for sample types annotations
SampleTypeMapping = {
    "Pulmón": "Tissue",
    "Lechón": "Tissue",
    "Fluido oral": "Fluid",
    "Pulmón/ Fluido oral": "Mixed",
    "Heces": "Feces",
    "Raspado traqueal": "Swab",
    "Hisopo/ Raspado traqueal/ Pulmón": "Mixed",
    "Intestino": "Tissue",
    "Hisopo": "Swab",
    "Raspado traqueal/ Pulmón": "Mixed",
    "Fluido oral/ Pulmón": "Mixed",
    "Heces/intestino": "Mixed",
    "Hisopo rectal": "Swab",
    "Porción intestinal": "Tissue",
    "Exudado": "Fluid",
    "Pulmón ": "Tissue",
    "Líquido pericárdico": "Fluid",
    "Raspado traqueal/ Fluido oral": "Mixed",
    "Heces/hisopo rectal": "Mixed",
    "Hisopo nasal": "Swab",
    "Hisopo nasal/ Pulmón/ Fluido Oral": "Mixed",
    "Órgano": "Tissue",
    "Hisopo intestinal": "Swab",
    "Feto": "Tissue",
    "Hisopo pulmonar": "Swab",
    "Heces/hisopos": "Mixed",
    "Ganglio": "Tissue",
    "Intestino/heces/hisopo": "Mixed",
    "Pulmón/ Fluido Oral": "Mixed",
    "Pulmón / Fluido Oral": "Mixed",
    "Hisopo fecal": "Swab",
    "Cabeza": "Tissue",
}
df["SampleType"] = df["SampleType"].map(SampleTypeMapping).fillna("Missing")



# FarmType Mapping for FarmTypes annotations
FarmTypeMapping = {

    'Madres reproductoras': 'Sow',
    'Engreíx': 'Finishing', #Fatening
    'Engreix': 'Finishing',
    'Engorde': 'Finishing',
}
# Map the 'FarmType' column using the FarmType mapping
df['PigStages_FarmType'] = df['FarmType'].map(FarmTypeMapping)

# PigStages mapping
PigStageMapping = {

    'lechones abortados': 'Other',
    'nan': 'Missing',
    '6-9 semanas': 'Nursery',
    '3m0s': 'Finishing',
    '1M3S': 'Weaning',
    '3M0S': 'Finishing',
    '40 kg': 'Finishing',
    '50 kg': 'Finishing',
    'cerda': 'Sow',
    '2,5 meses': 'Finishing',
    'recria': 'Finishing',
    'recría': 'Finishing',
    's/n': 'Missing',
    'transició': 'Nursery',
    'cebo': 'Finishing',
    'destete': 'Weaning',
    'primerizas destetadas': 'Sow',
    'primerizas': 'Gilt',
    'semanas 43 y 53': 'Gilt',
    'Cerdas maternidad': 'Sow',
    'cerdas gestantes antes parto': 'Sow',
    'cerdo de cebo': 'Finishing',
    '3 meses en cebo': 'Finishing',
    'final transición': 'Nursery',
    '3 semanas en cebo': 'Finishing',
    'fin de cebo-4 meses': 'Finishing',
    '1 mes estancia en cebo': 'Finishing',
    'cerdo 75 kg': 'Finishing',
    '2 meses estancia': 'Finishing',
    '14-18 semanas de vida': 'Finishing',
    '12 semanas de vida': 'Finishing',
    'final destete (6 semanas vida)': 'Weaning',
    '24 semanas de vida': 'Finishing',
    '16 semanas de vida': 'Finishing',
    'multíparas/ primerizas': 'Others',
    'cerdas gestantes': 'Sow',
    'cerda de recría': 'Gilt',
    'Transició': 'Nursery',
    'Cebo': 'Finishing',
    'Destete': 'Weaning',
    'Primerizas destetadas': 'Weaning',
    'Recría': 'Nursery',
    'Primerizas': 'Gilt',
    'Semanas 43 y 53': 'Unknown',
    'Cerdas gestantes antes parto': 'Sow',
    'Cerdo de cebo': 'Finishing',
    '3 meses en cebo': 'Finishing',
    'Final transición': 'Nursery',
    '3 semanas en cebo': 'Finishing',
    'Fin de cebo-4 meses': 'Finishing',
    '1 mes estancia en cebo': 'Finishing',
    'Cerdo 75 kg': 'Finishing',
    '2 meses estancia': 'Finishing',
    '14-18 semanas de vida': 'Nursery',
    'Final destete (6 semanas vida)': 'Weaning',
    '24 semanas de vida': 'Nursery',
    '16 semanas de vida': 'Nursery',
    'Multíparas/ primerizas': 'Gilt',
    'Cerdas gestantes': 'Sow',
    'Cerda de recría': 'Sow',
    '2 mesos': 'Finishing',
    '10 setmanes': 'Weaning',
    'Cerdas': 'Sow',
    'Nulíparas/Primerizas destetadas': 'Gilt',
    'nulíparas/Primerizas destetadas': 'Gilt',
    '10 semanas': 'Weaning',
    '1 mes cebo': 'Finishing',
    '3,5 meses cebo': 'Finishing',
    'Cerdas gestación': 'Sow',
    'Cerdas nulíparas': 'Gilt',
    '2 meses cebo': 'Finishing',
    '2,5 meses cebo': 'Finishing',
    '105 kg': 'Finishing',
    'Cerda lactante': 'Sow',
    '2 meses y medio estancia en cebo': 'Finishing',
    'Lechones 3-4 semanas postdestete': 'Weaning',
    '3 meses de cebo': 'Finishing',
    'Primeriza': 'Gilt',
    'primeriza': 'Gilt',
    '11 semanas': 'Weaning',
    '14 semanas': 'Nursery',
    '18 semanas': 'Nursery',
    '20 setmanes': 'Nursery',
    '10 días de vida': 'Suckling',
    'lechones 80 días': 'Finishing',
    'lechones 60 días': 'Finishing',
    'Lechones 80 días': 'Finishing',
    'Lechones 60 días': 'Finishing',
    '3 meses': 'Finishing',
    'Primerizas gestación': 'Gilt',
    'Gestación': 'Sow',
    'Transición': 'Nursery',
    'Gestants/Recria': 'Sow',
    'Gestants/ Recria': 'Sow',
    'gestants/Recria': 'Sow',
    'gestants/ Recria': 'Sow',
    '1,5M ': 'Finishing',
    '1,5M': 'Finishing',
    '1,5m': 'Finishing',
    '5 mesos': 'Finishing',
    'Truges': 'Sow',
    'truges': 'Sow',
    'Cerdas adultas': 'Sow',
    'cerdas adultas': 'Sow',
    'Cerdas adultas': 'Sow',
    'Cerdas reposición': 'Sow',
    'cerdas reposición': 'Sow',
    '1 mes estancia': 'Finishing',
    'Truja 2 anys': 'Sow',
    'truja 2 anys': 'Sow',
    'Cerda recría': 'Sow',
    'cerda recría': 'Sow',
    '4 mesos': 'Finishing',
    '3 mesos': 'Finishing',
    'Cerdo 25 semanas vida': 'Finishing',
    'Cerdo25 semanas vida': 'Finishing',
    'cerdo 25 semanas vida': 'Finishing',
    'Lechón lactante': 'Suckling',
    'lechón lactante': 'Suckling',
    '8-9 semanas': 'Weaning',
    '8-9 semanas vida': 'Weaning',
    '2M0S': 'Finishing',
    '2m0S': 'Finishing',
    '2m0s': 'Finishing',
    'Lechón': 'Suckling',
    'lechón': 'Suckling',
    '8 semanas cebo': 'Finishing',
    'Lechones 1 semana post-destete': 'Weaning', 
    'lechones 1 semana post-destete': 'Weaning',
    'Lechón destetado': 'Weaning',
    'Lechones/ Recría': 'Nursery',
    '4 semanas': 'Weaning',
    '3 meses cebo': 'Finishing',
    'Lechones': 'Suckling',
    'lechones': 'Suckling',
    'Transición': 'Nursery', #689
    'transición': 'Nursery',
    'Lechón 15 días post destete': 'Weaning',
    'lechón 15 días post destete': 'Weaning',
    'Lechónes 15 días post destete': 'Weaning',
    'Lechones 3-4 semanas': 'Weaning',
    'Lechones 7-8 semanas de vida': 'Weaning',
    'lechones 7-8 semanas de vida': 'Weaning',
    'Lechón transición': 'Weaning',
    'Lechón transición': 'Nursery',
    '1M3S': 'Finishing',
    '1,5 meses estancia': 'Finishing',
    'Llavora': 'Gilt',
    'llavora': 'Gilt',
    'Lechones 2 semanas de vida': 'Suckling',
    '8 semanas': 'Weaning',
    '4 semanas de vida': 'Weaning',
    '12 setmanes': 'Nursery',
    'Cerda primeriza': 'Gilt',
    'Lechones 2-3 semanas de vida': 'Suckling',
    'lechones 2-3 semanas de vida': 'Suckling',
    '6 semanas': 'Weaning',
    '5 semanas': 'Weaning',
    '4, 5 meses': 'Finishing',
    'Lechones 3 semanas post destete': 'Weaning',
    'lechones 3 semanas post destete': 'Weaning',
    'Lechones transición': 'Nursery',
    'lechones transición': 'Nursery',
    '4-5 semanas de vida': 'Weaning',
    'Cerdas primerizas': 'Gilt',
    'Cerdas maternidad': 'Sow',
    '2-3 semanas de vida': 'Suckling',
    'Lechones 16 kg': 'Nursery',
    'lechones 16 kg': 'Nursery',
    'lechon 16 kg': 'Nursery',
    'Lechones 3 semanas': 'Weaning',
    'lechones 3 semanas': 'Weaning',
    'lechon 3 semanas': 'Weaning',
    'Lechón 4 semanas post-destete': 'Weaning',
    'lechón 4 semanas post-destete': 'Weaning',
    'Lechónes 4 semanas post-destete': 'Weaning',
    'lechónes 4 semanas post-destete': 'Weaning',
    '4,5 meses': 'Finishing',
    '20 días cebo': 'Finishing',
    '10d': 'Suckling',
    'Cerdo 100 kg': 'Finishing',
    'cerdo 100 kg': 'Finishing',
    'Reproductoras': 'Sow',
    'reproductoras': 'Sow',
    '3,5 meses': 'Finishing',
    '6 setmanes de vida': 'Weaning',
    'Truja': 'Sow',
    'truja': 'Sow',
    '2 meses': 'Finishing',
    '2M3S': 'Finishing',
    '2m3s': 'Finishing',
    '2m3S': 'Finishing',
    '2M3s': 'Finishing',
    'Lechones 3,5 - 4 meses': 'Nursery',
    'lechones 3,5 - 4 meses': 'Nursery',
    'Lechones 3,5-4 meses': 'Nursery',
    '5 meses': 'Finishing',
    'Destete 2, 3 y 4 semanas, y cebo': 'Finishing',
    'destete 2, 3 y 4 semanas, y cebo': 'Finishing', #553
    'Lechones 4 semanas de vida': 'Weaning',
    'lechones 4 semanas de vida': 'Weaning',
    '5 semanas vida; transición': 'Nursery',
    'Lechones post destete': 'Weaning',
    'lechones post destete': 'Weaning',
    'Lechones 12-14 semanas': 'Nursery',
    'lechones 12-14 semanas': 'Nursery', #569
    '10-12 semanas': 'Nursery',
    'Lechones destete': 'Weaning',
    'lechones destete': 'Weaning',
    '6 semanas destetados': 'Weaning',
    '20 setmanes; recria': 'Nursery',
    '60 kg': 'Finishing',
    '30 días': 'Weaning',
    'Lechones 45 días': 'Weaning',
    'lechones 45 días': 'Weaning',
    'Cerdo 70 kg': 'Finishing',
    'Cerdo 60 kg': 'Finishing',
    'cerdo 60 kg': 'Finishing',
    'cerdo 70 kg': 'Finishing',
    '55 kg': 'Finishing',
    '7 semanas de vida': 'Weaning',
    'Cerdo 45 kg': 'Finishing',
    'cerdo 45 kg': 'Finishing',
    'Lechones 4 semanas post destete': 'Weaning',
    'lechones 4 semanas post destete': 'Weaning',
    'lechone 4 semanas post destete': 'Weaning',
    'Lechone 4 semanas post destete': 'Weaning',
    'Lechones 11-12 semanas de vida': 'Nursery',
    'lechones 11-12 semanas de vida': 'Nursery',
    'Cerdas 12 meses': 'Sow',
    'cerdas 12 meses': 'Sow',
    '80 kg': 'Finishing',
    '21 dies': 'Suckling',
    'Cerdos 30 kg': 'Finishing',
    'Cerdo 110 Kg': 'Finishing',
    'cerdos 30 kg': 'Finishing',
    'cerdo 110 Kg': 'Finishing',
    'Llavores': 'Gilt',
    'llavores': 'Gilt',
    'Cerda gestante': 'Sow',
    'cerda gestante': 'Sow',
    '6 setmanes': 'Weaning',
    'Destete Transición': 'Nursery',
    '30-35 Kg': 'Finishing',
    'Lechones 7-8 semanas de vida': 'Weaning',
    '40-60 kg': 'Finishing',
    'Lechones 30 kg': 'Finishing',
    'lechones 30 kg': 'Finishing',
    'Garrins destete 5-7 setmanes vida': 'Weaning',
    'garrins destete 5-7 setmanes vida': 'Weaning',
    ' Transición': 'Nursery',
    'Transición': 'Nursery',
    'Transición ': 'Nursery',
    'Precebo': 'Finishing',
    'precebo': 'Finishing',
    '50 dies': 'Suckling',
    '70 dies': 'Suckling',
    'Lechones 3 semanas de vida': 'Weaning',
    'lechones 3 semanas de vida': 'Weaning',
    '1 mes': 'Finishing',
    '3-4 setmanes': 'Weaning',
    'Cerdo 60 kg': 'Finishing',
    'Cerdo 105 kg': 'Finishing',
    'cerdo 105 kg': 'Finishing',
    'Gestación': 'other', #810
    'gestación': 'other',
    ' Gestación': 'other', 
    ' gestación': 'other', 
    'Nulíparas': 'Gilt', #841
    'nulíparas': 'Gilt', #841
    ' Nulíparas': 'Gilt', #841
    'Cerda maternidad': 'Sow', #841
    'cerda maternidad': 'Sow', #841
    'Lechones cebadero': 'Finishing',
    'lechones cebadero': 'Finishing', #765
    'Lechones 5 y 25 días de vida': 'Suckling',
    'lechones 5 y 25 días de vida': 'Suckling',
    '90 días': 'Finishing',
    '35 días transición': 'Nursery',
    '8 semanas vida': 'Weaning',
    'Truja parideres': 'Sow',
    'truja parideres': 'Sow',
    'Cerdo 105 Kg': 'Finishing',
    'Lechones 10 días post destete': 'Weaning',
    'lechones 10 días post destete': 'Weaning',
    '2,5 mesos': 'Finishing',
    '0M1S': 'Suckling',
    '0m1S': 'Suckling',
    '0M1s': 'Suckling',
    '0m1s': 'Suckling',
    '3 setmanes': 'Weaning',
    'Nulíparas': 'Gilt',
    'Cerda maternidad': 'Sow',
    '1 semana post-destete': 'Weaning',
    '9 semanas': 'Weaning',
    'Lechón 9 semanas': 'Weaning',
    'Madres': 'Sow',
    'madres': 'Sow',
    'Lechón Destete': 'Weaning',
    '>100d gestación': 'Sow',
    'Cerdo 110 kg': 'Finishing',
    'Cerdos en cebo': 'Finishing',
    'Paridera/Recría': 'Sow',
    'Parideras': 'Sow',
    'Lechón paridera': 'Suckling',
    'lechón paridera': 'Suckling',
    '6 mesos': 'Finishing',
    'Cerdas': 'Sow',
    'cerdas': 'Sow',
    'Recría': 'Gilt',
    'Transición': 'Nursery',
    'Paridera/Recría': 'Sow',
    '10 semanas': 'Nursery',
    'Parideras': 'Farrowing',
    'Cerda 18 meses': 'Sow',
    'Entrada cebo': 'Finishing',
    'transició': 'Nursery',
    '4 setmanes': 'Weaning',
    '35 kg': 'Finishing',
    'Cerda nulípara': 'Gilt',
    'Lechones paridera': 'Suckling',
    'Lechón destete': 'Weaning',
    '5 setmanes': 'Weaning',
    '12 semanas de vida': 'Nursery',
    'Cerda maternidad': 'Sow',
}

df['PigStages_Age'] = df['Age'].str.lower().map(PigStageMapping)

#Determine the PigStage by Age, Age Unit, FarmType. and map to one column
# Combine the results from both mappings (PigStages_Age and PigStages_FarmType)
#df['PigStages'] = df.apply(lambda x: x['PigStages_Age'] if pd.notna(x['PigStages_Age']) else x['PigStages_FarmType'], axis=1)
#This code checks each row in the DataFrame `df` and returns the value of `'PigStages_Age'` 
#if it's not `NaN`, otherwise it returns the value of `'PigStages_FarmType'`. 
#If both are `NaN`, it fills the result with `"Missing"`

df['PigStage'] = df.apply(lambda x: x['PigStages_Age'] if pd.notna(x['PigStages_Age']) else x['PigStages_FarmType'], axis=1).fillna("Missing")

# Province Mapping and annotations
# Province to Region (Autonomous Community) Mapping
ProvinceMapping = {
    'Huesca': 'Huesca',  
    'Zaragoza': 'Zaragoza', 
    'Lleida': 'Lleida',  
    'Treuel': 'Teruel',  # Corrected spelling
    'Barcelona': 'Barcelona',
    'Navarra': 'Navarre',
    'Teruel': 'Teruel',
    'Tarragona': 'Tarragona',
    'Málaga': 'Malaga', 
    'Castellón': 'Castellon',  # Remove accent
    'Almería': 'Almeria',  # Remove accent
    'Ourense': 'Ourense', 
    'Soria': 'Soria', 
    'Girona': 'Girona', 
    'Múrcia': 'Murcia', 
    'Granada': 'Granada', 
    'Albacete': 'Albacete',
    'Castelló': 'Castello',  # Remove accent
    'La Rioja': 'LaRioja',
}
df["Province"] = df["Province"].map(ProvinceMapping).fillna("Missing")

# Convert 'Date' column to datetime format
#df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
# Format 'Date' column as 'DD-MM-YYYY'
#df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')

# Drop intermediate columns if needed
df.drop(['PigStages_Age', 'PigStages_FarmType'], axis=1, inplace=True)



In [45]:
df.head (50)

,SampleNumber,Province,Date,Pathogen,Result,SampleType,Age,FarmType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,S-21-0055,Missing,2021-01-05,Swine influenza virus,1,Tissue,NaN,NaN,PCR,Spain,Meat,1,Missing
1,S-21-0058,Huesca,2021-01-05,Swine influenza virus,0,Tissue,Lechones abortados,NaN,PCR,Spain,Meat,1,Other
2,S-21-0062,Missing,2021-01-05,Swine influenza virus,0,Tissue,NaN,NaN,PCR,Spain,Meat,1,Missing
3,S-21-0067,Missing,2021-01-05,Swine influenza virus,0,Tissue,NaN,NaN,PCR,Spain,Meat,1,Missing
4,S-21-0102,Missing,2021-01-08,Swine influenza virus,0,Fluid,NaN,NaN,PCR,Spain,Meat,1,Missing
5,S-21-0106,Missing,2021-01-08,Swine influenza virus,0,Fluid,6-9 semanas,NaN,PCR,Spain,Meat,1,Nursery
6,S-21-0156,Huesca,2021-01-12,Swine influenza virus,1,Tissue,NaN,Madres reproductoras,PCR,Spain,Meat,1,Sow
7,S-21-0223,Missing,2021-01-14,Swine influenza virus,0,Fluid,NaN,NaN,PCR,Spain,Meat,1,Missing
8,S-21-0270,Missing,2021-01-15,Swine influenza virus,0,Mixed,NaN,NaN,PCR,Spain,Meat,1,Missing
9,S-21-0274,Missing,2021-01-15,Swine influenza virus,0,Fluid,NaN,NaN,PCR,Spain,Meat,1,Missing


In [46]:
#df.to_csv('output/PigLab1CopyTest2.csv', index=False)

In [47]:

df = df.drop(['Age'], axis=1)

df = df.drop(['FarmType'], axis=1)

In [48]:
df.head (50)


,SampleNumber,Province,Date,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,S-21-0055,Missing,2021-01-05,Swine influenza virus,1,Tissue,PCR,Spain,Meat,1,Missing
1,S-21-0058,Huesca,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Other
2,S-21-0062,Missing,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Missing
3,S-21-0067,Missing,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Missing
4,S-21-0102,Missing,2021-01-08,Swine influenza virus,0,Fluid,PCR,Spain,Meat,1,Missing
5,S-21-0106,Missing,2021-01-08,Swine influenza virus,0,Fluid,PCR,Spain,Meat,1,Nursery
6,S-21-0156,Huesca,2021-01-12,Swine influenza virus,1,Tissue,PCR,Spain,Meat,1,Sow
7,S-21-0223,Missing,2021-01-14,Swine influenza virus,0,Fluid,PCR,Spain,Meat,1,Missing
8,S-21-0270,Missing,2021-01-15,Swine influenza virus,0,Mixed,PCR,Spain,Meat,1,Missing
9,S-21-0274,Missing,2021-01-15,Swine influenza virus,0,Fluid,PCR,Spain,Meat,1,Missing


In [49]:
barometer = df

In [51]:
barometer

,SampleNumber,Province,Date,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,S-21-0055,Missing,2021-01-05,Swine_influenza_virus,1,Tissue,PCR,Spain,Meat,1,Missing
1,S-21-0058,Huesca,2021-01-05,Swine_influenza_virus,0,Tissue,PCR,Spain,Meat,1,Other
2,S-21-0062,Missing,2021-01-05,Swine_influenza_virus,0,Tissue,PCR,Spain,Meat,1,Missing
3,S-21-0067,Missing,2021-01-05,Swine_influenza_virus,0,Tissue,PCR,Spain,Meat,1,Missing
4,S-21-0102,Missing,2021-01-08,Swine_influenza_virus,0,Fluid,PCR,Spain,Meat,1,Missing
...,...,...,...,...,...,...,...,...,...,...,...
1039,S-23-9748,Huesca,2023-10-24,Lawsonia_intracellularis,1,Feces,PCR,Spain,Meat,1,Nursery
1040,S-23-9778,Missing,2023-10-25,Mycoplasma_hyopneumoniae,0,Tissue,PCR,Spain,Meat,1,Missing
1041,S-23-9852,Missing,2023-10-26,Mycoplasma_hyopneumoniae,0,Tissue,PCR,Spain,Meat,1,Missing
1042,S-23-9921,Missing,2023-10-27,Mycoplasma_hyopneumoniae,0,Tissue,PCR,Spain,Meat,1,Nursery


### Step 02: Create an RDF graph and namespaces.

In [52]:
g = rdflib.Graph()
xsd = Namespace('http://www.w3.org/2001/XMLSchema#')
g.bind('xsd', xsd)

# Define your custom namespace for your ontology's properties
LHO = Namespace("http://www.purl.org/decide/LiveStockHealthOnto/LHO#")
g.bind('LHO', LHO)

SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
g.bind('skos', SKOS)

decide=  Namespace("http://www.purl.org/decide#")
g.bind('decide', decide)
ncit = Namespace("http://purl.obolibrary.org/obo/NCIT_C25464")
g.bind('ncit', ncit)
agrovoc = Namespace ("http://aims.fao.org/aos/agrovoc")
g.bind('agrovoc', agrovoc)

###  Step 03: Iterate over the Panda DataFrame and map to ontology properties:

In [57]:
#For Pig Data frame RDF conversion
#for index, row in barometer.head(500).iterrows():
for index, row in barometer.iterrows():
    SampleNumber = row["SampleNumber"]
    Date = row["Date"]
    Pathogen = row["Pathogen"]
    Breed = row["Breed"]
    Country = row["Country"]
    Province = row["Province"]
    SampleType = row["SampleType"]
    SampleResult = row["Result"]
    DiagnosticTest = row["DiagnosticTest"]
    PigStage= row["PigStage"]

    
    # Create a unique URI for each sample based on the row index
    PigSample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab1PigSample_{index}")
    
    # Add sample type assertion
    g.add((PigSample_uri, RDF.type, LHO.PigSample))
    description = "An individual representing a sample from a Pig and Piglets."
    g.add((PigSample_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "Pathogen" in row and row["Pathogen"]:
        Pathogen = row["Pathogen"]
        Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen}")
        g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        description = "An individual representing Pig pathogen."
        g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Breed" in row and row["Breed"]:
        Breed = row["Breed"]
        Breed_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Breed}")
        g.add((PigSample_uri, LHO.hasBreed, Breed_uri))
        g.add((Breed_uri, RDF.type, LHO.Breed))
        description = "An individual representing a Piglet breed which is Meat."
        g.add((Breed_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "DiagnosticTest" in row and row["DiagnosticTest"]:
        DiagnosticTest= row["DiagnosticTest"]
        DiagnosticTest_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{DiagnosticTest}")
        g.add((PigSample_uri, LHO.hasDiagnosticTest, DiagnosticTest_uri))
        g.add((DiagnosticTest_uri, RDF.type, LHO.DiagnosticTest))
    if DiagnosticTest == "PCR":
        description = "An individual representing DNA/RNA amplification for rapid pathogen detection."
    elif SampleType == "Culture":
        description = "An individual representing a Growing live microorganisms for identification."
        g.add((DiagnosticTest_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Country" in row and row["Country"]:
        Country = row["Country"].strip()  # Use strip() to remove leading and trailing spaces
        # Encode the Country value to create a valid URI
        Country = urllib.parse.quote(Country)
        Country_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Country}")
    
        # Map to the specific class URI in your ontology and add the label
        g.add((PigSample_uri, LHO.hasCountry, Country_uri))
        g.add((Country_uri, RDF.type, URIRef("http://purl.obolibrary.org/obo/NCIT_C25464")))  # Use the specific class URI
        g.add((Country_uri, RDFS.label, Literal("Spain")))
        description = "An individual representing different Countries."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Province" in row and row["Province"]:
        Province = row["Province"].strip()
        Province = urllib.parse.quote(Province)
        Province_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Province}")
        g.add((PigSample_uri, LHO.hasProvince, Province_uri))
        g.add((Province_uri, RDF.type, LHO.Province))
        description = "An individual representing different Province."
        g.add((Province_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "PigStage" in row and row["PigStage"]:
        Stage = row["PigStage"]
        Stage_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Stage}")
        g.add((PigSample_uri, LHO.hasProductionStages, Stage_uri))
        g.add((Stage_uri, RDF.type, LHO.PigProductionStage))
        description = "An individual representing Production stage ."
        g.add((Stage_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "SampleType" in row and row["SampleType"]:
        SampleType = row["SampleType"]
        SampleType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleType}")
        # Add statements for SampleType
        g.add((PigSample_uri, LHO.hasSampleType, SampleType_uri))
        g.add((SampleType_uri, RDF.type, URIRef("http://www.purl.org/decide#SampleType")))
    if SampleType == "Autopsy":
        description = "An individual representing a cattle sample obtained through autopsy."
    elif SampleType == "BAL":
        description = "An individual representing a cattle sample obtained through bronchoalveolar lavage (BAL)."
    elif SampleType == "SWAB":
        description = "An individual representing a cattle sample obtained through swabbing."
    elif SampleType == "Missing":
        description = "An individual representing a missing or unspecified cattle sample type."
        g.add((SampleType_uri, RDFS.comment, Literal(description, lang="en")))
           
    if "Result" in row and row["Result"]:
        SampleResult = row["Result"]
        # Create a unique URI for the Result based on the value
        Result_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleResult}")
        g.add((PigSample_uri, LHO.hasResult, Result_uri))
        g.add((Result_uri, RDF.type, decide.SampleResult))
    if SampleResult == "1":
        description = "An individual representing a Positive test result"
    elif SampleResult == "0":
        description = "An individual representing a negative test result"
    elif SampleResult == "missing":
        description = "An individual representing an unknown or missing test result."
    g.add((Result_uri, RDFS.comment, Literal(description, lang="en")))


    if "Date" in row and not pd.isnull(row["Date"]):  # Check for NaN or None values
        Date = str(row["Date"]).strip()
    
    # Split the datetime string and take the date part
        Date = Date.split()[0]
    
    # Create a unique URI for the Date based on the value (use the Date directly)
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
    
    # Add Date assertion
        g.add((PigSample_uri, LHO.hasDate, Date_uri))
        g.add((Date_uri, RDF.type, LHO.Date))
    

# Serialize the RDF graph to a file
rdf_output_file = "output/PigLab1.rdf"
g.serialize(rdf_output_file, format="xml")

# Serialize the RDF graph to Turtle format and print it too see RDF conversion
turtle_data = g.serialize(format="turtle")
print(turtle_data)

@prefix LHO: <http://www.purl.org/decide/LiveStockHealthOnto/LHO#> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix decide: <http://www.purl.org/decide#> .
@prefix decide1: <https://www.purl.org/decide#> .
@prefix ncit: <http://purl.obolibrary.org/obo/NCIT_C25464> .
@prefix obo: <http://purl.obolibrary.org/obo/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://aims.fao.org/aos/agrovoc/c_6145> a owl:Class ;
    rdfs:label "Poultry"@en ;
    obo:IAO_0000412 "https://agrovoc.fao.org/browse/agrovoc/en/page/c_6145" ;
    rdfs:comment "All domesticated birds used for the production of meat or eggs for consumption, for the production of other commercial products, for restocking, supplies of game, or for breeding these categories of birds." ;
    rdfs:sub

### Step 4: Load the RDF data and ontology into a Panda DataFrame: 

In [56]:
# Parse the ontology file in OWL format and add it to the graph
path_to_ontology = "Ontology/LivestockHealthOntology.owl"
g.parse(path_to_ontology, format="xml")


<Graph identifier=N2bd69500dca84634aa9f2dd858f424e3 (<class 'rdflib.graph.Graph'>)>

### Step 05: Query the data from updated ontology 

In [60]:

query = """
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO: <http://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT
  (strafter(str(?Sample), "#") AS ?SampleName)
  (strafter(str(?Pathogen), "#") AS ?PathogenName)
  (strafter(str(?Breed), "#") AS ?BreedName)
  (strafter(str(?SampleType), "#") AS ?SampleTypeName)
  (strafter(str(?DiagnosticTest), "#") AS ?DiagnosticTestName)
  (strafter(str(?PigProductionStages), "#") AS ?ProductionStage)
  (strafter(str(?Province), "#") AS ?ProvinceName)
  (strafter(str(?Country), "#") AS ?CountryName)
  (strafter(str(?SampleResult), "#") AS ?SampleResultName)

WHERE {
 {
  ?Sample rdf:type LHO:PigSample .
  ?Sample LHO:hasPathogen ?Pathogen .
  ?Sample LHO:hasBreed ?Breed .
  ?Sample LHO:hasSampleType ?SampleType .
  FILTER (?SampleType = LHO:Tissue)
  ?Sample LHO:hasProvince ?Province .
  ?Sample LHO:hasCountry ?Country .
  ?Sample LHO:hasProductionStages ?PigProductionStages .
  FILTER (?PigProductionStages = LHO:Finishing)
  ?Sample LHO:hasResult ?SampleResult .
  FILTER (?SampleResult = LHO:1)
  ?Sample LHO:hasDiagnosticTest ?DiagnosticTest .
  }
}
"""


# Execute the query and retrieve the results
results = g.query(query)

# Convert the results to a Pandas dataframe
data = []
for row in results:
    data.append(list(row))

df = pd.DataFrame(data, columns=[
    "Sample", "Pathogen", "Breed", "SampleType", 
    "DiagnosticTest","PigProductionStage", "Province", "Country", "SampleResult"
])

# Display the dataframe
df


,Sample,Pathogen,Breed,SampleType,DiagnosticTest,PigProductionStage,Province,Country,SampleResult
0,Lab1PigSample_12,Swine_influenza_virus,Meat,Tissue,PCR,Finishing,Missing,Spain,1
1,Lab1PigSample_45,Swine_influenza_virus,Meat,Tissue,PCR,Finishing,Huesca,Spain,1
2,Lab1PigSample_86,Mycoplasma_hyopneumoniae,Meat,Tissue,PCR,Finishing,Missing,Spain,1
3,Lab1PigSample_90,Lawsonia_intracellularis,Meat,Tissue,PCR,Finishing,Missing,Spain,1
4,Lab1PigSample_200,Mycoplasma_hyopneumoniae,Meat,Tissue,PCR,Finishing,Missing,Spain,1
5,Lab1PigSample_222,Mycoplasma_hyopneumoniae,Meat,Tissue,PCR,Finishing,Missing,Spain,1
6,Lab1PigSample_224,Mycoplasma_hyopneumoniae,Meat,Tissue,PCR,Finishing,Missing,Spain,1
7,Lab1PigSample_229,Mycoplasma_hyopneumoniae,Meat,Tissue,PCR,Finishing,Missing,Spain,1
8,Lab1PigSample_279,Mycoplasma_hyopneumoniae,Meat,Tissue,PCR,Finishing,Missing,Spain,1
9,Lab1PigSample_307,Mycoplasma_hyopneumoniae,Meat,Tissue,PCR,Finishing,Missing,Spain,1
